## A simple swarm

In [69]:
%load_ext autoreload 
%autoreload 2
import sys
sys.tracebacklimit = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
from core.agent import *
from core.world import * 
import numpy as np
import random

In [71]:
world = World((100, 100))
swarm = [Agent() for _ in range(1000)]
for agent in swarm:
    world.add_agent(agent)

In [72]:
swarm = initialize_positions_randomly(world, swarm)

In [73]:
# Test movements by doing random actions
for agent in swarm:
    choice = np.random.randint(1, 5)
    agent.move(choice)

world.update()